In [76]:
import pandas as pd
import yfinance as yf
import sqlalchemy
import datetime
import os
import time
import pandas_ta as ta
from backtesting import Backtest, Strategy
import numpy as np
from backtesting.lib import resample_apply
from backtesting.test import GOOG
from pandas_datareader import data

In [106]:
start_date = '2020-01-12'
end_date = '2023-01-11'
df=yf.download('AAPL',start_date, end_date,interval='1d',auto_adjust=True )
df.index.names = ['']


[*********************100%***********************]  1 of 1 completed


In [107]:
df

,Open,High,Low,Close,Volume
,,,,,
2020-01-13,76.326688,77.656599,76.206673,77.629654,121532000
2020-01-14,77.565971,77.779050,76.456487,76.581390,161954400
2020-01-15,76.378108,77.272062,75.814789,76.253197,121923600
2020-01-16,76.804274,77.321058,76.436895,77.208389,108829200
2020-01-17,77.460642,78.065593,77.149597,78.063148,137816400
...,...,...,...,...,...
2023-01-04,126.889999,128.660004,125.080002,126.360001,89113600
2023-01-05,127.129997,127.769997,124.760002,125.019997,80962700
2023-01-06,126.010002,130.289993,124.889999,129.619995,87686600


In [68]:
GOOG

,Open,High,Low,Close,Volume
2004-08-19,100.00,104.06,95.96,100.34,22351900
2004-08-20,101.01,109.08,100.50,108.31,11428600
2004-08-23,110.75,113.48,109.05,109.40,9137200
2004-08-24,111.24,111.60,103.57,104.87,7631300
2004-08-25,104.96,108.00,103.88,106.00,4598900
...,...,...,...,...,...
2013-02-25,802.30,808.41,790.49,790.77,2303900
2013-02-26,795.00,795.95,784.40,790.13,2202500
2013-02-27,794.80,804.75,791.11,799.78,2026100
2013-02-28,801.10,806.99,801.03,801.20,2265800


In [100]:
import datetime
import pandas_ta as ta
import pandas as pd

from backtesting import Backtest
from backtesting.lib import crossover, TrailingStrategy

def sup1(fr):
    return ta.supertrend(fr.High, fr.Low, fr.Close, length=10, multiplier=1.0)
def sup2(fr):
    return ta.supertrend(fr.High, fr.Low, fr.Close, length=10, multiplier=2.0)
def sup3(fr):
    return ta.supertrend(fr.High, fr.Low, fr.Close, length=10, multiplier=3.0)


class RsiOscillator(Strategy):

    # Do as much initial computation as possible
    def init(self):
        
        #super().init()
        #super().set_trailing_sl(1)
        
        self.atr= self.I(ta.atr, high= self.data.High.s,low= self.data.Low.s, 
                            close= self.data.Close.s, length=10)
        self.super1 = self.I(ta.supertrend, high= self.data.High.s,low= self.data.Low.s, 
                             close= self.data.Close.s, length=10, multiplier=1.0, plot=False, overlay=True)
        self.super2 = self.I(ta.supertrend, high= self.data.High.s,low= self.data.Low.s, 
                             close= self.data.Close.s, length=10, multiplier=2.0,  plot=False, overlay=True)
        self.super3 = self.I(ta.supertrend, high= self.data.High.s,low= self.data.Low.s, 
                             close= self.data.Close.s, length=10, multiplier=3.0,  plot=False, overlay=True)
        self.super1_w = resample_apply(
            'W-FRI',sup1, self.data.df,plot=False)
        self.super2_w = resample_apply(
            'W-FRI',sup2, self.data.df, plot=False)
        self.super3_w = resample_apply(
            'W-FRI',sup3, self.data.df,plot=False)        
        self.ema20 = self.I(ta.ema, close= self.data.Close.s, length=20, plot=True)
        self.ema20_w = resample_apply('W-FRI',ta.ema, self.data.Close.s, length=20, plot=False)
        self.ema50 = self.I(ta.ema, close= self.data.Close.s, length=50, plot=True)
        self.ema50_w = resample_apply('W-FRI',ta.ema, self.data.Close.s, length=50, plot=False)
        self.macd = self.I(ta.macd, close= self.data.Close.s, plot=False)
        self.macd_w = resample_apply('W-FRI',ta.macd, self.data.Close.s, plot=False)
    # Step through bars one by one
    # Note that multiple buys are a thing here
    def next(self):
        
        #super().next()
        price = self.data.Close.s
        low= self.data.Low.s
        atr = self.atr
        super1=self.super1[0]
        super2=self.super2[0]
        super3=self.super3[0]
        super1w=self.super1_w[0]
        super2w=self.super2_w[0]
        super3w=self.super3_w[0]
        ema20= self.ema20
        ema50= self.ema50
        ema20w= self.ema20_w
        ema50w= self.ema50_w
        macdh=self.macd[1]
        macdh_w=self.macd_w[1]

        if  not self.position and price[-2]<ema20[-2] and price[-1]>ema20[-1]:
                self.position.close()
                self.buy()
        elif price[-2]>ema20[-2] and price[-1]<ema20[-1]:
                self.position.close()
                #self.sell()

bt = Backtest(df, RsiOscillator, cash=5000, commission=.002)

stats = bt.run()
#stats
bt.plot(plot_volume=False, plot_pl=False)

Row(id='30537', ...)

In [61]:
from backtesting import Strategy
from backtesting.lib import crossover

def SMA(values, n):
    """
    Return simple moving average of `values`, at
    each step taking into account `n` previous values.
    """
    return pd.Series(values).rolling(n).mean()


class SmaCross(Strategy):
    # Define the two MA lags as *class variables*
    # for later optimization
    n1 = 10
    n2 = 20
    
    def init(self):
        # Precompute the two moving averages
        self.sma1 = self.I(SMA, self.data.Close, self.n1)
        self.sma2 = self.I(SMA, self.data.Close, self.n2)
    
    def next(self):
        # If sma1 crosses above sma2, close any existing
        # short trades, and buy the asset
        if crossover(self.sma1, self.sma2):
            self.position.close()
            self.buy()

        # Else, if sma1 crosses below sma2, close any existing
        # long trades, and sell the asset
        elif crossover(self.sma2, self.sma1):
            self.position.close()
            #self.sell()
bt = Backtest(df, SmaCross, cash=10_000, commission=.002)
stats = bt.run()
stats

Start                     2020-01-13 00:00:00
End                       2023-01-10 00:00:00
Duration                   1093 days 00:00:00
Exposure Time [%]                   58.278146
Equity Final [$]                 17480.716766
Equity Peak [$]                  24317.391803
Return [%]                          74.807168
Buy & Hold Return [%]                68.40217
Return (Ann.) [%]                   20.492543
Volatility (Ann.) [%]               28.891918
Sharpe Ratio                         0.709283
Sortino Ratio                        1.316286
Calmar Ratio                         0.697745
Max. Drawdown [%]                  -29.369682
Avg. Drawdown [%]                    -4.40372
Max. Drawdown Duration      372 days 00:00:00
Avg. Drawdown Duration       41 days 00:00:00
# Trades                                   16
Win Rate [%]                             50.0
Best Trade [%]                      80.599826
Worst Trade [%]                    -13.504434
Avg. Trade [%]                    

In [ ]:
help(ta.macd)

In [28]:
deneme=ta.macd(df.Close)
deneme

,MACD_12_26_9,MACDh_12_26_9,MACDs_12_26_9
2020-01-13,NaN,NaN,NaN
2020-01-14,NaN,NaN,NaN
2020-01-15,NaN,NaN,NaN
2020-01-16,NaN,NaN,NaN
2020-01-17,NaN,NaN,NaN
...,...,...,...
2023-01-05,-4.906985,-0.712439,-4.194546
2023-01-06,-4.575295,-0.304599,-4.270696
2023-01-09,-4.221005,0.039753,-4.260758
2023-01-10,-3.849056,0.329361,-4.178417


In [13]:
deneme2= ta.supertrend(df.High, df.Low, df.Close, length=10, multiplier=1.0)
deneme2

,SUPERT_10_1.0,SUPERTd_10_1.0,SUPERTl_10_1.0,SUPERTs_10_1.0
2019-01-03,0.000000,1,NaN,NaN
2019-01-04,NaN,1,NaN,NaN
2019-01-07,NaN,1,NaN,NaN
2019-01-08,NaN,1,NaN,NaN
2019-01-09,NaN,1,NaN,NaN
...,...,...,...,...
2022-10-12,228.163041,-1,NaN,228.163041
2022-10-13,227.764242,-1,NaN,227.764242
2022-10-14,227.764242,-1,NaN,227.764242
2022-10-17,227.764242,-1,NaN,227.764242


In [8]:
help(ta.ema)

Help on function ema in module pandas_ta.overlap.ema:

ema(close, length=None, talib=None, offset=None, **kwargs)
    Exponential Moving Average (EMA)
    
    The Exponential Moving Average is more responsive moving average compared to the
    Simple Moving Average (SMA).  The weights are determined by alpha which is
    proportional to it's length.  There are several different methods of calculating
    EMA.  One method uses just the standard definition of EMA and another uses the
    SMA to generate the initial value for the rest of the calculation.
    
    Sources:
        https://stockcharts.com/school/doku.php?id=chart_school:technical_indicators:moving_averages
        https://www.investopedia.com/ask/answers/122314/what-exponential-moving-average-ema-formula-and-how-ema-calculated.asp
    
    Calculation:
        Default Inputs:
            length=10, adjust=False, sma=True
        if sma:
            sma_nth = close[0:length].sum() / length
            close[:length - 1] = n

In [12]:
ema=ta.ema(close=df.Close, length=20)
ema

2020-01-13           NaN
2020-01-14           NaN
2020-01-15           NaN
2020-01-16           NaN
2020-01-17           NaN
                 ...    
2022-01-05    173.813966
2022-01-06    173.546684
2022-01-07    173.320955
2022-01-10    173.118618
2022-01-11    173.209200
Name: EMA_20, Length: 505, dtype: float64

In [16]:
bbands = ta.bbands(close = df.Close, std = 1)
bbands

,BBL_5_1.0,BBM_5_1.0,BBU_5_1.0,BBB_5_1.0,BBP_5_1.0
2019-01-03,NaN,NaN,NaN,NaN,NaN
2019-01-04,NaN,NaN,NaN,NaN,NaN
2019-01-07,NaN,NaN,NaN,NaN,NaN
2019-01-08,NaN,NaN,NaN,NaN,NaN
2019-01-09,20.726126,21.691867,22.657607,8.904171,0.953952
...,...,...,...,...,...
2022-10-12,215.799657,223.580005,231.360353,6.959789,0.092563
2022-10-13,217.449548,220.298004,223.146461,2.586003,0.749608
2022-10-14,210.328390,216.682004,223.035618,5.864459,-0.420106
2022-10-17,210.181759,215.960004,221.738248,5.351217,0.793342


In [57]:
help(ta.atr)

Help on function atr in module pandas_ta.volatility.atr:

atr(high, low, close, length=None, mamode=None, talib=None, drift=None, offset=None, **kwargs)
    Average True Range (ATR)
    
    Averge True Range is used to measure volatility, especially volatility caused by
    gaps or limit moves.
    
    Sources:
        https://www.tradingview.com/wiki/Average_True_Range_(ATR)
    
    Calculation:
        Default Inputs:
            length=14, drift=1, percent=False
        EMA = Exponential Moving Average
        SMA = Simple Moving Average
        WMA = Weighted Moving Average
        RMA = WildeR's Moving Average
        TR = True Range
    
        tr = TR(high, low, close, drift)
        if 'ema':
            ATR = EMA(tr, length)
        elif 'sma':
            ATR = SMA(tr, length)
        elif 'wma':
            ATR = WMA(tr, length)
        else:
            ATR = RMA(tr, length)
    
        if percent:
            ATR *= 100 / close
    
    Args:
        high (pd.Series):

In [14]:
atr=ta.atr(df.Close, df.High, df.Low, length=10)
atr

2020-01-13         NaN
2020-01-14         NaN
2020-01-15         NaN
2020-01-16         NaN
2020-01-17         NaN
                ...   
2022-01-05    4.601081
2022-01-06    4.469069
2022-01-07    4.270720
2022-01-10    3.989800
2022-01-11    4.287774
Name: ATRr_10, Length: 505, dtype: float64

In [49]:
from backtesting import Strategy
from backtesting.lib import crossover

def SMA(values, n):
    """
    Return simple moving average of `values`, at
    each step taking into account `n` previous values.
    """
    return pd.Series(values).rolling(n).mean()


class SmaCross(Strategy):
    # Define the two MA lags as *class variables*
    # for later optimization
    n1 = 10
    n2 = 20
    
    def init(self):
        # Precompute the two moving averages
        self.sma1 = self.I(SMA, self.data.Close, self.n1)
        self.sma2 = self.I(SMA, self.data.Close, self.n2)
    
    def next(self):
        # If sma1 crosses above sma2, close any existing
        # short trades, and buy the asset
        if crossover(self.sma1, self.sma2):
            self.position.close()
            self.buy()

        # Else, if sma1 crosses below sma2, close any existing
        # long trades, and sell the asset
        elif crossover(self.sma2, self.sma1):
            self.position.close()
            #self.sell()
bt = Backtest(df, SmaCross, cash=10_000, commission=.002)
stats = bt.run()
stats

Start                     2020-01-13 00:00:00
End                       2023-01-10 00:00:00
Duration                   1093 days 00:00:00
Exposure Time [%]                   58.278146
Equity Final [$]                 17480.717027
Equity Peak [$]                  24317.390466
Return [%]                           74.80717
Buy & Hold Return [%]                68.40217
Return (Ann.) [%]                   20.492543
Volatility (Ann.) [%]               28.891921
Sharpe Ratio                         0.709283
Sortino Ratio                        1.316286
Calmar Ratio                         0.697745
Max. Drawdown [%]                  -29.369677
Avg. Drawdown [%]                   -4.403719
Max. Drawdown Duration      372 days 00:00:00
Avg. Drawdown Duration       41 days 00:00:00
# Trades                                   16
Win Rate [%]                             50.0
Best Trade [%]                      80.599829
Worst Trade [%]                    -13.504434
Avg. Trade [%]                    

In [50]:
bt.plot(plot_volume=False, plot_pl=False)

Row(id='23551', ...)

In [ ]:
        if (price[-1]>ema20w[-1] or price[-1]>ema50w[-1])\
        and (price[-1]>super1w[-1] or price[-1]>super2w[-1] or price[-1]>super3w[-1]):
            if  price[-2]<super1[-2] and price[-1]>super1[-1]:
                #self.position.close()
                #self.buy(size=1, sl=price[-1]-2*atr[-1])
                self.buy(size=1)
        if price[-1]<ema50[-1]:
                self.position.close()
                #self.sell()